## Extractive text summarization

In [22]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
nltk.download('punkt') # one time execution
import re
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
!pip install --upgrade nltk

Requirement already up-to-date: nltk in /usr/local/lib/python3.7/dist-packages (3.6.1)


In [26]:
nltk.download("stopwords") 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:


!pip install datasets
from datasets import load_metric
metric = load_metric("meteor")

install wikihowAll.csv from this url  https://ucsb.app.box.com/s/ap23l8gafpezf4tq3wapr6u8241zz358 and put it in your drive . Then mount your drive to colab in following cell

In [14]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [15]:
'''
This code is used to create article and summary files from the csv file.
The output of the file will be a directory of text files representing seoarate articles and their summaries.
Each summary line starts with tag "@summary" and the article is followed by "@article".
'''

import os


# read data from the csv file (from the location it is stored)
Data = pd.read_csv(r'/content/drive/MyDrive/wikihowAll.csv')
Data = Data.astype(str)
rows, columns = Data.shape

# create a file to record the file names. This can be later used to divide the dataset in train/dev/test sets
title_file = open('titles.txt', 'wb')

# The path where the articles are to be saved
path = "articles"
if not os.path.exists(path): os.makedirs(path)

# go over the all the articles in the data file
for row in range(rows):
    abstract = Data.loc[row,'headline']      # headline is the column representing the summary sentences
    article = Data.loc[row,'text']           # text is the column representing the article

    #  a threshold is used to remove short articles with long summaries as well as articles with no summary
    if len(abstract) < (0.75*len(article)):
        # remove extra commas in abstracts
        abstract = abstract.replace(".,",".")
        abstract = abstract.encode('utf-8')
        # remove extra commas in articles
        article = re.sub(r'[.]+[\n]+[,]',".\n", article)
        article = article.encode('utf-8')
        

        # a temporary file is created to initially write the summary, it is later used to separate the sentences of the summary
        with open('temporaryFile.txt','wb') as t:
            t.write(abstract)
        
        # file names are created using the alphanumeric charachters from the article titles.
        # they are stored in a separate text file.
        filename = Data.loc[row,'title']
        filename = "".join(x for x in filename if x.isalnum())
        filename1 = filename + '.txt'
        filename = filename.encode('utf-8')
        title_file.write(filename+b'\n')

        
        with open(path+'/'+filename1,'wb') as f:
            # summary sentences will first be written into the file in separate lines
            with open('temporaryFile.txt','r') as t:
                for line in t:
                    line=line.lower()
                    if line != "\n" and line != "\t" and line != " ":
                        f.write(b'@summary'+b'\n')
                        f.write(line.encode('utf-8'))
                        f.write(b'\n')
                    
            # finally the article is written to the file
            f.write(b'@article' + b'\n')    
            f.write(article)

title_file.close()

I- Naive implementation : <br>
Test text rank on one article

In [16]:
f = open('./articles/HowtoWinHisHeart.txt','r')
txt = f.read()
txt = txt.replace('@summary','')
pair = re.split('@article',txt)
pair = list(map(lambda x : re.sub(r"\'s", " \'s", x),pair))
pair = list(map(lambda x :re.sub(r"n\'t", " n\'t", x),pair))
pair = list(map(lambda x : re.sub(r"\s{2,}", " ", x), pair))
pair = list(map(lambda x : x.strip(r"\n"),pair))
pair = list(map(lambda x : x.strip(),pair))
pair = list(map(lambda x : x.replace('\n',''),pair))
pair

['be confident. smile. place yourself in his path. play it cool. make him laugh. be nice to his friends. use the "3 strikes and you\'re out" rule. have a successful first date. pay attention to his interests. do n\'t take him for granted. maintain some space. decide if he \'s truly worth it. tell him how you feel.',
 'The most important way you can attract the guy of your dreams is to be confident and content in your own self. You can do this by wearing a cute outfit, doing something new with your hair, trying a new makeup style, or simply just thinking positively, because confidence is one of the most attractive traits a person can possibly have and it will make you less anxious about failure, which makes it hard to take the leap and into a potential relationship. Do n\'t judge yourself or others harshly. When you focus on all your flaws that is all that you\'ll think you have to offer and when you focus on the flaws of others, you are making your own brain toxic with negative thought

Split to sentences

In [17]:
from nltk.tokenize import sent_tokenize
import random
sentences = sent_tokenize(pair[1])
sentences_summary = sent_tokenize(pair[0])
sentences = sentences + sentences_summary
sentences
random.shuffle(sentences)

Get word embeddings

In [18]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2021-04-17 19:11:36--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-04-17 19:11:36--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-04-17 19:11:36--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

Extract word vectors

In [19]:
# Extract word vectors

word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [20]:
len(word_embeddings)

400000

Additional preprocessing

In [27]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(sen):
   sen_new = " ".join([i for i in sen if i not in stop_words])
   return sen_new
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
clean_sentences

['might feel way feel okay',
 'show life outside thinking',
 'n understand interests n even necessarily enjoy interested enough talk guy show interested',
 'flat saying glad life good easy way make sure knows value little things show appreciate',
 'way two together much talk',
 'even better make eye contact chosen sweetheart give little smile',
 'good way let someone know interested care take interest interests opinions things',
 'pay attention interests',
 'n ever feel like need fix make better',
 'however n mean need hide interest pretend interested',
 'gotten past flirting stage need first date',
 'make laugh',
 'mention time mistook stranger best friend slapped butt way date missed step bus fell door front everyone',
 'give little chance talk one another without bunch people around like party n time think stalking',
 'given openings ask date even coffee n gone times stop trying move',
 'maintain relationships friends',
 'n pretend expert talking',
 'n pretty good sign either narcis


Compute word embedding of each sentence as mean of the word embeddings of words

In [28]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

Compute similarity matrix between two sentences


In [29]:
# similarity matrix
from sklearn.metrics.pairwise import cosine_similarity
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]


Use graph theory's algorithm Page rank to rank sentences

In [30]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

Now u can specify the number of sentences u want in your summary and print ur summary

In [31]:
sn = 2
for i in range(sn):
    print(ranked_sentences[i][1])

Flat out saying "I am so glad you're in my life" is a good and easy way to make sure he knows that you value him.Do some little things that show you appreciate him.
You do n't want to enter into a co-dependent, ca n't-breath-without-you kind of a relationship, as much as the movies make it look nice.


Show ground truth summary

In [32]:
summary_sents = sent_tokenize(pair[0])
summary_sents

['be confident.',
 'smile.',
 'place yourself in his path.',
 'play it cool.',
 'make him laugh.',
 'be nice to his friends.',
 'use the "3 strikes and you\'re out" rule.',
 'have a successful first date.',
 'pay attention to his interests.',
 "do n't take him for granted.",
 'maintain some space.',
 "decide if he 's truly worth it.",
 'tell him how you feel.']

2- Optimized implementation

link : https://github.com/summanlp/textrank

In [33]:
!pip install summa

     |████████████████████████████████| 61kB 4.4MB/s 
  Created wheel for summa: filename=summa-1.2.0-cp37-none-any.whl size=54411 sha256=6d37c12c04df8e8b3991c30f6d4c8b60c547c2558bdec4ba727e2d00eb9a4825
  Stored in directory: /root/.cache/pip/wheels/6a/09/68/e2f2861c01d86407c3fa5220826ed7eed2abaa56b001be5970
Successfully built summa


In [34]:
from summa.summarizer import summarize
text = pair[0]+pair[1]
predicted_summary = summarize(text, ratio=0.05)

In [35]:
#ground truth summary 
true_summary = pair[0]

In [36]:
!pip install rouge-score

In [38]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeL'], use_stemmer=False)
scores = scorer.score(true_summary,
                      predicted_summary)

Function to Run TextRank on a number of articles and compute scores : ROUGE1, ROUGE2, ROUGEL, METEOR

In [51]:
directory = "./articles"
def compute_scores(ratio,n):
  ## n : number of articles on which we test textrank example : 10000
  ## ratio : summary size percentage of article example :  0.2
  mean_rouge1 = 0
  mean_rougeL = 0
  mean_rouge2 = 0
  mean_meteor = 0
  n_articles = 0
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeL'], use_stemmer=False)
  ### Loop over files in articles directory, predict summaries with textrank and compute scores
  f = open('titles.txt','r')
  for line in f.readlines():
      n_articles+=1
      w = open(directory+'/'+line.rstrip()+'.txt','r')
      txt = w.read()
      w.close()
      txt = txt.replace('@summary','')
      pair = re.split('@article',txt)
      pair = list(map(lambda x : re.sub(r"\'s", " \'s", x),pair))
      pair = list(map(lambda x :re.sub(r"n\'t", " n\'t", x),pair))
      pair = list(map(lambda x : re.sub(r"\s{2,}", " ", x), pair))
      pair = list(map(lambda x : x.strip(r"\n"),pair))
      pair = list(map(lambda x : x.strip(),pair))
      pair = list(map(lambda x : x.replace('\n',''),pair))
      text = pair[0]+pair[1]
      predicted_summary = summarize(text, ratio=ratio)
      true_summary = pair[0]
      scores = scorer.score(true_summary,
                        predicted_summary)
      score2 = metric.compute(predictions=[predicted_summary], references=[true_summary])

      mean_rouge1 += scores['rouge1'].fmeasure
      mean_rougeL += scores['rougeL'].fmeasure
      mean_rouge2 += scores['rouge2'].fmeasure
      mean_meteor += score2['meteor']
      if n_articles == n :
          break

  mean_rouge1 = mean_rouge1/n_articles
  mean_rougeL = mean_rougeL/n_articles
  mean_rouge2 = mean_rouge2/n_articles
  mean_meteor = mean_meteor/n_articles
  f.close()
  return(mean_rouge1,mean_rouge2,mean_rougeL,mean_meteor)

In [52]:
compute_scores(0.05,1000)

(0.1943390271285373,
 0.0702391474544891,
 0.1259226341923782,
 0.1397042998370221)

In [53]:
compute_scores(0.1,1000)

(0.27339272444900065,
 0.11615919501938982,
 0.1800780650891979,
 0.2373434146881059)

In [54]:
compute_scores(0.2,1000)

(0.26416101568272954,
 0.13528260471963324,
 0.1779353885647301,
 0.2874180843335402)

In [55]:
compute_scores(0.05,10000)

(0.17784605670415424,
 0.06197578506020388,
 0.11353365363588588,
 0.12194088068425424)

In [56]:
compute_scores(0.1,10000)

(0.27383158147561965,
 0.11648594149182079,
 0.17956909117734007,
 0.22833840469835404)

In [57]:
compute_scores(0.2,10000)

(0.27680398361042363,
 0.14322503454683883,
 0.18697595989870253,
 0.292527880586423)